<a href="https://colab.research.google.com/github/mateusribeirocampos/diollm/blob/main/Facial_recognition_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import PIL
from PIL import Image
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
from os import listdir
from torchvision.io import read_image
import torchvision.transforms as T
import numpy as np
from glob import glob
import shutil
import cv2
import random
from facenet_pytorch import InceptionResnetV1

In [ ]:
image_size = 140
batch_size = 20

In [ ]:
files = glob("celebrities_images\*")
files

In [ ]:
classes = ["Aamir Khan", "Abhay Deol", "Abhishek Bachchan", "Aishwarya Rai", "Ajay Devgn"]
aamir_files = glob('celebrities_images\\Aamir Khan\*')

In [ ]:
def crop_faces(files):
    for file in files:
        img = cv2.imread(file)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')
        faces = cascade.detectMultiScale(gray)
        if len(faces) > 0:
            for i, (x, y, w, h) in enumerate(faces):
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 2)
                face = img[y:y + h, x:x + w]
                color_face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                cv2.imwrite(f'face{random.randint(0,200)}.jpg', color_face)

In [ ]:
crop_faces(aamir_files)

In [ ]:
abhay_files = glob('celebrities_images\\Abhay Deol\*')

In [ ]:
abhishek_files = glob('celebrities_images\\Abhishek Bachchan\*')
crop_faces(abhishek_files)

In [ ]:
aishwarya_files = glob('celebrities_images\\Aishwarya Rai\*')
crop_faces(aishwarya_files)

In [ ]:
ajay_files = glob('celebrities_images\\Ajay Devgn\*')
crop_faces(ajay_files)

In [ ]:
def train_test_split(files, path):
    files_train = files[:70]
    files_test = files[70:]
    for file in files_test:
        shutil.move(file, path)

In [ ]:
aamir_files = glob('celebrities_images\\train\\Aamir Khan\*')
train_test_split(aamir_files, 'celebrities_images\\test\\Aamir Khan')

In [ ]:
abhay_files = glob('celebrities_images\\train\\Abhay Deol\*')
train_test_split(abhay_files,'celebrities_images\\test\\Abhay Deol' )

In [ ]:
abhishek_files = glob('celebrities_images\\train\\Abhishek Bachchan\*')
train_test_split(abhishek_files, 'celebrities_images\\test\\Abhishek Bachchan')

In [ ]:
aishwarya_files = glob('celebrities_images\\train\\Aishwarya Rai\*')
train_test_split(aishwarya_files, 'celebrities_images\\test\\Aishwarya Rai')

In [ ]:
ajay_files = glob('celebrities_images\\train\\Ajay Devgn\*')
train_test_split(ajay_files,'celebrities_images\\test\\Ajay Devgn' )

In [ ]:
Data transforms and loaders
data_dir = "celebrities_images/"
train_transforms = transforms.Compose([transforms.Resize(image_size),
                                       transforms.CenterCrop(image_size),
                                       transforms.RandomRotation(60),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(image_size),
                                      transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


In [ ]:
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

In [ ]:
valid_size = 0.1
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

In [ ]:
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size= batch_size,
                                          sampler = train_sampler, drop_last= True)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size= batch_size,
                                          sampler = valid_sampler, drop_last= True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size= batch_size, drop_last=True)
print(len(train_loader))
print(len(valid_loader))
print(len(test_loader))

In [ ]:
img, label = next(iter(train_loader))
print(img.shape)
print(label.shape)
print(img.min())
print(img.max())

In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    plt.imshow(np.transpose(img, (1, 2, 0)))

In [ ]:
fig = plt.figure(figsize=(50, 4))

for idx in np.arange(5):
    ax = fig.add_subplot(2, 20, idx+1, xticks=[], yticks=[])
    imshow(img[idx])
    ax.set_title(classes[label[idx]])

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.arc = InceptionResnetV1(pretrained='vggface2',
                                    classify = True,
                                    num_classes= len(classes))
    def forward(self, img):
        logits = self.arc(img)
        return logits

In [ ]:
model = Model()
print(model)

In [ ]:
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
epochs = 20

In [ ]:
valid_loss_min = np.Inf
train_accuracy, val_accuracy = [],[]
for epoch in range(epochs):
    train_loss = 0.0
    valid_loss = 0.0
    t_acc = 0.0
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        train_loss += loss.item()
        preds = F.log_softmax(logits, dim=1)
        ps = torch.exp(preds)
        top_k, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        t_acc += equals.sum().item()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        model.eval()
        v_acc = 0.0
        for images, labels in valid_loader:
            logits = model(images)
            loss = criterion(logits, labels)
            valid_loss += loss.item()
            preds = F.log_softmax(logits, dim=1)
            ps = torch.exp(preds)
            top_k, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            v_acc += equals.sum().item()
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    train_accuracy.append(t_acc/len(train_loader.sampler))
    val_accuracy.append(v_acc/len(valid_loader.sampler))

    print("Epoch ", epoch+1, "train loss ", train_loss, "validation loss ", valid_loss)

    if valid_loss <= valid_loss_min:
        print("validation loss decreased", valid_loss_min, "---->", valid_loss)
        print("Saving the model")
        torch.save(model.state_dict(), "model_facial_rec.pt")
        valid_loss_min = valid_loss

In [ ]:
model.load_state_dict(torch.load("model_facial_rec.pt"))

In [ ]:
plt.plot(train_accuracy, label="Training Accuracy")
plt.plot(val_accuracy, label="Validation Accuracy")
plt.legend()

In [ ]:
test_loss = 0.0
class_correct = list(0. for i in range(len(classes)))
class_total = list(0. for i in range(len(classes)))

model.eval()

for data, target in test_loader:
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    p = F.log_softmax(output, dim=1)
    _, pred = torch.max(p, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())
    for i in range(len(target)):

        label = target.data[i]

        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(4):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

In [ ]:
dataiter = iter(test_loader)
images, labels = next(dataiter)
images.numpy()

output = model(images)
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())

fig = plt.figure(figsize=(50, 15))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title("{} ({})".format(classes[preds[idx]], classes[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))